In [43]:
from textblob import TextBlob
import tweepy
import re
import pandas as pd

In [44]:
accessToken="1272053768067010562-fQTNkoduS347TE8qzpLh8mxiYcB8TC"
accessTokenSecret="8yBxCzXyKobVV42fewzBtjbXNXGmtzTuPUg61ZaJyqZoz"
consumerKey="QDj1hUwKFp3Hnux2muCSoB4B9"
consumerKeySecret="ZRA0FRiAPzNKghZDyvP93BZYRu7fC6maiTrWmXPL349e0Lorv6"

In [45]:
auth=tweepy.OAuthHandler(consumerKey,consumerKeySecret)
auth.set_access_token(accessToken,accessTokenSecret)
api=tweepy.API(auth,wait_on_rate_limit=True)

In [46]:
def cleanText(text):
    text=re.sub('https?://[^\s]+','',text)
    text=re.sub('@[^\s]+','',text)
    text=re.sub(r'#','',text)
    text=re.sub(r'RT[\s]','',text)
    text=re.sub('\n',"",text)
    return text

In [47]:
numOfTweets= 1000
searchWord="covid"
date_since="2020-07-01"
tweets=tweepy.Cursor(api.search, q=searchWord, lang='en',since=date_since,tweet_mode="extended").items(numOfTweets)

In [48]:
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [49]:
def convert(Polarity):
    if(Polarity==0):
        return 0
    elif(Polarity>0.01):
        return 1
    else:
        return -1

In [50]:
df=pd.DataFrame([tweet.full_text for tweet in tweets],columns=['Tweets'])
df['Tweets']=df['Tweets'].apply(cleanText)
df['Polarity']=df['Tweets'].apply(getPolarity)
df['label']=df['Polarity'].apply(convert)
df.drop_duplicates(keep=False,inplace=True) 

In [51]:
print(df)

                                                Tweets  Polarity  label
6    To help slow the spread of COVID-19,  is partn...  0.050000      1
8     fr tho🥳i still be thinking bout those damn ta...  0.700000      1
9     Assistant professor Shana Stoddard is helping...  0.000000      0
11    NEW: White House public tours to start again ...  0.045455      1
12      Hahahaha! Where he’s being drafted I LOVE g...  0.058571      1
..                                                 ...       ...    ...
991   “​There was nobody coming, there was no help ...  0.000000      0
993   💥💥SHOCK REPORT:Up to 90% of COVID-positive Am...  0.000000      0
996   Statement from Wirral’s Director for Public H...  0.200000      1
997  NEW  GUEST BLOG: What will RevenueRecovery req...  0.001263     -1
999   Nicola Sturgeon announces a £25m transition t...  0.500000      1

[526 rows x 3 columns]


In [52]:
df.to_excel("covid5.xls")